In [ ]:
import os
import pandas as pd
import tensorflow
import numpy as np
from sklearn import preprocessing
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
%matplotlib inline

from numpy.random import seed


from keras.layers import Input, Dropout
from keras.layers.core import Dense 
from keras.models import Model, Sequential, load_model
from keras import regularizers
from keras.models import model_from_json

                                                                    #Data Preprocessing  
data_dir = r'D:\brazildata\normal'
merged_data = pd.DataFrame()
counter = 1
for filename in os.listdir(data_dir):#each for loop reads millions of data points from each fault catagory
    #print(filename)
    dataset=pd.read_csv(os.path.join(data_dir, filename), sep=',')
    #print(dataset)

    # tmp_dataset_squared = np.square(dataset)
    # tmp_dataset_mean = np.array(tmp_dataset_squared.mean())
    # dataset_mean_abs = np.sqrt(tmp_dataset_mean)

    dataset_mean_abs = np.array(dataset.abs().mean())
    dataset_mean_abs = pd.DataFrame(dataset_mean_abs.reshape(1,8))
    #dataset_mean_abs.index = [filename]
    filename = filename[:-4]
    dataset_mean_abs['FaultType'] = 0                        #Add manual Label based on data folder
    counter = counter+1
    
    merged_data = merged_data.append(dataset_mean_abs)

data_dir = r'D:\brazildata\imbalance\6g'
#merged_data = pd.DataFrame()
#print(counter)
for filename in os.listdir(data_dir):
    #print(filename)
    dataset=pd.read_csv(os.path.join(data_dir, filename), sep=',')
    #print(dataset)

    # tmp_dataset_squared = np.square(dataset)
    # tmp_dataset_mean = np.array(tmp_dataset_squared.mean())
    # dataset_mean_abs = np.sqrt(tmp_dataset_mean)

    dataset_mean_abs = np.array(dataset.abs().mean())
    dataset_mean_abs = pd.DataFrame(dataset_mean_abs.reshape(1,8))
    #dataset_mean_abs.index = [filename]
    filename = filename[:-4]
    dataset_mean_abs['FaultType'] = 1
    counter = counter+1
    merged_data = merged_data.append(dataset_mean_abs)

data_dir = r'D:\brazildata\overhang\ball_fault\0g'
#merged_data = pd.DataFrame()

for filename in os.listdir(data_dir):
    #print(filename)
    dataset=pd.read_csv(os.path.join(data_dir, filename), sep=',')
    #print(dataset)

    # tmp_dataset_squared = np.square(dataset)
    # tmp_dataset_mean = np.array(tmp_dataset_squared.mean())
    # dataset_mean_abs = np.sqrt(tmp_dataset_mean)

    dataset_mean_abs = np.array(dataset.abs().mean())
    dataset_mean_abs = pd.DataFrame(dataset_mean_abs.reshape(1,8))
    #dataset_mean_abs.index = [filename]
    filename = filename[:-4]
    dataset_mean_abs['FaultType'] = 2
    counter = counter+1
    merged_data = merged_data.append(dataset_mean_abs)

#print(counter)
num_cols = ['Bearing 1']
merged_data.columns = ['Bearing 1-1','Bearing 1-2','Bearing 2-1','Bearing 2-2', 'Bearing 3-1','Bearing 3-2','Bearing 4-1','Bearing 4-2', 'FaultType']
merged_data.reset_index(drop=True, inplace=True)
#merged_data.index = pd.to_datetime(merged_data.index, format='%Y.%m.%d.%H.%M.%S')
#merged_data.index = pd.to_datetime(merged_data.index, format='UTC')
#merged_data = merged_data.sort_index()
merged_data.to_csv('Imb1.csv')
#merged_data.index.names = ['index']
#merged_data.head()
num_cols = ['Bearing 1-1','Bearing 1-2','Bearing 2-1','Bearing 2-2', 'Bearing 3-1','Bearing 3-2','Bearing 4-1','Bearing 4-2']

dataset_train = merged_data #['2004-02-12 11:02:39':'2004-02-13 23:52:39']
#dataset_test = merged_data['2004-02-13 23:52:39':]
#dataset_train.plot(merged_data.loc[:,"date"], merged_data.loc[:,"Bearing 1"], figsize = (12,6))
#plt.plot(merged_data["FaultType"], merged_data[num_cols])
#plt.xticks(rotation=70)
#Importing the necessary packages and libaries
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import numpy as np

dataset = merged_data.values
scaler = StandardScaler()
X = dataset[:,0:8].astype(float)
X = scaler.fit_transform(X)
Y = dataset[:,8].astype(int)
# print(type(Y[0]))

# X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8)

neighbors = np.arange(1, 10)
train_accuracy = np.empty(len(neighbors))
test_accuracy = np.empty(len(neighbors))
 
# Loop over K values
for i, k in enumerate(neighbors):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred=knn.predict(X_test)
    # Compute training and test data accuracy
    train_accuracy[i] = knn.score(X_train, y_train)
    test_accuracy[i] = knn.score(X_test, y_test)
    cm_knn = confusion_matrix(y_test, y_pred)
    print('Confusion matrix & classification report for K=', k)
    print(cm_knn)
    print(classification_report(y_test, y_pred))
# Generate plot
plt.plot(neighbors, test_accuracy, label = 'Testing dataset Accuracy')
plt.plot(neighbors, train_accuracy, label = 'Training dataset Accuracy')
 
plt.legend()
plt.xlabel('n_neighbors')
plt.ylabel('Accuracy')
plt.show()
